# Домашнее задание 1. 

-  "Оптимальность" наивного Байесовского классификатора
- Оценка "важности" признаков (feature importance)

Для тех кто не знаком с библиотекой pandas я рекомендую вот эти два практических туториала:

-  https://nbviewer.jupyter.org/github/justmarkham/pandas-videos/blob/master/pandas.ipynb (1-3 часа на прохождение, 30 вопросов с видео разборами)
- https://github.com/guipsamora/pandas_exercises/ 11 топиков покрывающих весь основной функционал библиотеки


Кондрашов Кирилл Павлович

## 1. Наивный Байес

Ваша задача сравнить вероятности получаемые наивным байесовским классификатором с вероятностями получаемыми если не делать предположения о независимости признаков. Мы будем делать это для 2 признаков рассмотренных на семинаре: Пол и Класс билета (пассажира).

In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('./titanic_data.csv')

In [8]:
X = pd.DataFrame()

X['Sex'] = df['Sex'].map({
    'Male':0,
    'Female':1
})
X['PassengerClass'] = df['Passenger Class'].map({
    'First': 1,
    'Second': 2,
    'Third': 3
})

X['Survived'] = df['Survived'].map({
    'Yes': 1,
    'No': 0
})

y = X['Survived']
X = X.drop(['Survived'], axis=1)

In [9]:
def concat_Xy(X, y):
    data = X.copy()
    data['Survived'] = y
    return data

In [10]:
data = concat_Xy(X, y)

In [11]:
print("Statistics by 'Sex' feature")
sex_prob = data[['Sex', 'Survived']].groupby('Sex').mean()
print(sex_prob)

Statistics by 'Sex' feature
     Survived
Sex          
0    0.190985
1    0.727468


In [12]:
print("Statistics by 'PassengerClass' feature")
class_prob = data[['PassengerClass', 'Survived']].groupby('PassengerClass').mean()
print(class_prob)

Statistics by 'PassengerClass' feature
                Survived
PassengerClass          
1               0.619195
2               0.429603
3               0.255289


In [13]:
print("Naive statistics by both features")
naive_probs = class_prob.drop('Survived', axis=1)
naive_probs[0] = class_prob['Survived']
naive_probs[1] = class_prob['Survived']
mul_dict = {
    0: sex_prob.at[0, 'Survived'], 
    1: sex_prob.at[1, 'Survived']
}
naive_probs = naive_probs.mul(mul_dict)
print(naive_probs)

Naive statistics by both features
                       0         1
PassengerClass                    
1               0.118257  0.450444
2               0.082048  0.312522
3               0.048756  0.185715


In [14]:
print("Real statistics by both features")
real_probs = naive_probs.copy()
for row in real_probs.axes[0]:
    for col in real_probs.axes[1]:
        total = data.loc[
            (data['PassengerClass']==row) & 
            (data['Sex']==col)
        ]
        survived = total.loc[data['Survived']==1]
        real_probs.at[row, col] = survived.size / total.size
print(real_probs)

Real statistics by both features
                       0         1
PassengerClass                    
1               0.340782  0.965278
2               0.146199  0.886792
3               0.152130  0.490741


In [15]:
print("Difference")
print(naive_probs - real_probs)
print("MAE")
bias = np.mean(np.array(naive_probs - real_probs))
mse = np.mean(np.array(naive_probs - real_probs)**2)
print(bias)
print("MSE")
print(mse)

Difference
                       0         1
PassengerClass                    
1              -0.222525 -0.514833
2              -0.064151 -0.574270
3              -0.103374 -0.305026
MAE
-0.2973633086700544
MSE
0.12536659076999057


## 1.1 Разделим данные на тренировочную и тестовую части 70-30

сколько наблюдений оказалось в тренировочной и тестовой выборках?

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, random_state=5)

In [18]:
print(f"There are {X_train.shape[0]} observations in Train dataset and {X_test.shape[0]} in test")

There are 916 observations in Train dataset and 393 in test


In [19]:
X_train.describe()


,Sex,PassengerClass
count,916.000000,916.000000
mean,0.348253,2.272926
std,0.476677,0.854284
min,0.000000,1.000000
25%,0.000000,1.000000
50%,0.000000,3.000000
75%,1.000000,3.000000
max,1.000000,3.000000


In [20]:
X_test.describe()

,Sex,PassengerClass
count,393.000000,393.000000
mean,0.374046,2.346056
std,0.484492,0.796895
min,0.000000,1.000000
25%,0.000000,2.000000
50%,0.000000,3.000000
75%,1.000000,3.000000
max,1.000000,3.000000


In [21]:
y_train.describe()

count    916.000000
mean       0.385371
std        0.486949
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64

In [22]:
y_test.describe()

count    393.000000
mean       0.374046
std        0.484492
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64

Выборки достаточно стратифицированы. 37% и 39% выживших, 35% и 37% женщин.

## 1.2 Реализуйте функцию `get_probability_survived_bayes`

Которая считает честные вероятности выживания при условии заданного класса билета и пола:

$$
P(\text{y}|\text{Sex}, \text{Passenger Class}) = \frac{p(\text{Sex}|\text{Passenger Class}, \text{y}) p(\text{Passenger Class}|\text{ y}) p(\text{y})}{p(\text{Sex}|\text{Passenger Class}) p(\text{Passenger Class})}
$$

## Помните что оценка вероятностей всегда должна происходить на тренировчной выборке.

In [23]:
from typing import Tuple

In [24]:
# Hint 1. Используйте pandas.groupby
# Hint 2. Вы возвращаете tuple p_not_survived, p_survived, в котором
# вероятность выжить находится на второй позиции, чтобы когда вы берете аргмакс получать метки классов 0/1 (не выжил/выжил)
train_data = concat_Xy(X_train, y_train)


# Firstly I forgot about groupby
def get_probability_survived_bayes_no_groupby(pclass: int, sex: int) -> Tuple[float, float]:
    '''Given `Passenger Class` and `Sex` returns true probabilities tuple(p_not_survived, p_survived)'''
    cnt_sex_and_class_and_y = train_data.loc[
        (train_data['Sex']==sex) &
        (train_data['PassengerClass']==pclass) &
        (train_data['Survived']==1)
    ].shape[0]
    
    cnt_sex_and_class = train_data.loc[
        (train_data['Sex']==sex) &
        (train_data['PassengerClass']==pclass)
    ].shape[0]
    
    # Given formula reduces to one fraction. easy
    p = cnt_sex_and_class_and_y / cnt_sex_and_class
    return (1-p, p)


def get_probability_survived_bayes(pclass: int, sex: int) -> Tuple[float, float]:
    '''Given `Passenger Class` and `Sex` returns true probabilities tuple(p_not_survived, p_survived)'''
    p = train_data.groupby(['Sex', 'PassengerClass']).get_group((sex, pclass))['Survived'].mean()
    return (1-p, p)

In [25]:
print("Real statistics for train data")
real_probs = naive_probs.copy()
for row in real_probs.axes[0]:
    for col in real_probs.axes[1]:
        real_probs.at[row, col] = get_probability_survived_bayes(row, col)[1]
print(real_probs)

Real statistics for train data
                       0         1
PassengerClass                    
1               0.358209  0.963303
2               0.122807  0.863636
3               0.146132  0.541667


## 1.3 Реализуйте функцию `get_probability_survived_bayes_naive`

Которая оценивает вероятность выжить в предположении о независимости признаков

$$
P(\text{y}|\text{Sex}, \text{Passenger Class}) = \frac{p(\text{Sex}|\text{y}) p(\text{Passenger Class}|\text{ y}) p^2(\text{y})}{p(\text{Sex}) p(\text{Passenger Class})}
$$

In [61]:
def get_probability_survived_bayes_naive(pclass: int, sex: int) -> Tuple[float, float]:
    '''Given `Passenger Class` and `Sex` returns tuple(p_not_survived, p_survived), 
    under naive assumption of features independence'''
    p_survived = train_data['Survived'].mean()
    p_sex = train_data.value_counts('Sex', normalize=True)[sex]
    p_class = train_data.value_counts('PassengerClass', normalize=True)[pclass]
    p_sex_if_y = train_data.groupby('Survived').value_counts(['Sex'], normalize=True)[:, sex]
    p_class_if_y = train_data.groupby('Survived').value_counts(['PassengerClass'], normalize=True)[:, pclass]
    def p_for_y(y):
        p_y = p_survived if y == 1 else 1 - p_survived
        return p_y * p_y * p_sex_if_y[y] * p_class_if_y[y] / (p_sex * p_class)
    return (p_for_y(0), p_for_y(1))
        
    
# Hint 1. Поскольку вы делаете предположении о независимости признаков, то p(y=1|...) + p(y=0| ...) некорректно отнормированы
# и не будут суммироваться в 1. Не забудьте выполнить необходимую нормировку.

In [62]:
print("Naive statistics for train data")
real_probs = naive_probs.copy()
for row in real_probs.axes[0]:
    for col in real_probs.axes[1]:
        real_probs.at[row, col] = get_probability_survived_bayes_naive(row, col)[1]
print(real_probs)

Naive statistics for train data
                       0         1
PassengerClass                    
1               0.119176  0.473703
2               0.074660  0.296761
3               0.049528  0.196863


## 1.4 Используйте ваши функции для того чтобы сделать предсказание на тесте


Для всех 6 уникальных комбинаций значений признаков сравните вероятности принадлежности классу 1
 - насколько они отличаются?
 - влияют ли эти отличия на предсказание класса объекта, при пороге $p = 0.5$?

In [28]:
from tqdm.notebook import tqdm

In [29]:
y_pred_proba = []

for _, (sex, pclass, _) in X_test.iterrows():
    y_pred_proba.append(get_probability_survived_bayes_naive(pclass, sex))

ValueError: not enough values to unpack (expected 3, got 2)

## 1.5 Обучите имплементацию `MultinomialNB` из `sklearn`

Тот же вопрос что и 1.4, прокомментируйте отличия в оценке вероятности ваших имлементаций и имплементации
`MultinomialNB`. Влияют ли они на предсказание классов при стандартном пороге вероятности (p=0.5)? Заполните таблицу `probabilities`.

In [ ]:
probabilities = pd.DataFrame()
probabilities['Sex'] = [0,0,0,1,1,1]
probabilities['Passenger Class'] = [1,2,3,1,2,3]
probabilities['Bayes'] = [0]*6
probabilities['BayesNaive'] = [0]*6
probabilities['MultinomialNB'] = [0]*6

probabilities

,Sex,Passenger Class,Bayes,BayesNaive,MultinomialNB
0,0,1,0,0,0
1,0,2,0,0,0
2,0,3,0,0,0
3,1,1,0,0,0
4,1,2,0,0,0
5,1,3,0,0,0


## 2.1 Используя MultinomialNB обучите классификатор на бОльшем количестве признаков

1. Подготовьте признаки 
- 'Passenger Class',
- 'Sex',
- 'Age',
- 'No of Siblings or Spouses on Board',
- 'No of Parents or Children on Board',
- 'Port of Embarkation'

для обучения:
- закодируйте столбец `Port of Embarkation` по аналогии с `Passenger Class`
- заполните пропуски в столбце `Age` средним возрастом в датасете используйте `pd.DataFrame.fillna(...)`
- заполните пропуске в столбце `Port of Embarkation` самым частым портом посадки
- для разбиения на тренировчную и тестовую часть данных используйте `train_size=0.66` и `random_state=5`

2. обучите MultinomialNB модель на тренировочной части данных
2. Как изменилась точность вашей модели на тестовой части данных по сравнению с моделью обученной на 2 признаках?
3. Что означает признак `Life Boat`? Следуют ли на ваш взгляд включать его в список признаков для обучения модели? 
- Опишите сценарий (использования модели) в котором его следует включать в список признаков. 
- Опишите сценарий (использования модели) в котором его не следует включать в список признаков.
5. Какая ошибка содержится в предложенной последовательности действий в п. 2.1? 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=5)

## 2.2. Оценка важности признаков

Мы воспользуемся простым (и не очень хорошим^[1]) методом оценки важности признаков:

1. Воспользуйтесь моделью из п.2. Качество модели на тестовой части данных это ваш бейзлайн
2. Исключая по 1 признаку из `X` обучайте модель `MultinomialNB` и измеряйте ее точность на тестовой части данных
3. Сохраняйте величину падения или роста качества по сравнению с бейзлайном. Если точность упала значит признак "полезный", если выросло - "вредный". Абсолютная величина падения или роста в данном случае не так важна, важен только порядок признаков. Такая оценка называется качественной (в противовес количественной).
4. Прокомментируйте полученный результат, насколько он согласуется с вашей интуицией?
5. В чем на ваш взляд могут быть преимущества использования мЕньшего числа признаков при обучении моделей? (не обязательно в контексте данной задачи)


[1] а впрочем какие из них хороши?

jupyter notebook c вашим решением необходимо отправить на почту kurmukovai@gmail.com, с темой письма `[ml-ds-2024-ha1-Surname]`, например `[ml-ds-2023-Kurmukov]` до 12:00 МСК 08.02.2023. Задания отправленные до полуночи Вторника 06.02.2023 получают +1 балл в случае полного выполнения (не обязательно верного:))